In [ ]:
import keras.backend as K
import numpy as np
from keras.models import Model,load_model
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid


## load test data and loc_map

In [ ]:
test = np.load("checkpt/test_data.npy")
loc_map = np.load("checkpt/test_loc_map.npy")
test_label = np.loadtxt("checkpt/test_label.txt")

In [ ]:
test_label.shape

## read checkpoint

In [ ]:
model = load_model("checkpt/ckt/checkpt_0.h5")

In [ ]:
model.summary()

In [ ]:
pred = model.predict(test)

## get True Positive 

In [ ]:
## .argmax(axis =1 ) will return the biggest value of the two as 1, and the other as 0. i.e. [0.6 ,0.9] will give [0,1]
## this is a good format as our test_label is organised in [0,1] or [1,0] format.

In [ ]:
TP = np.where(pred.argmax(axis=1) == test_label.argmax(axis=1))

In [ ]:
## I will suggest to access the confidence of the predication. Usually we want 0.9 at least

In [ ]:
def return_heatmap(model, org_img, normalise=True):
    ## CAM code implementation
    ## we need to extract the last conv layer, and that depends on your architecture.
    test_img = model.output[:, 1]
    last_conv_layer = model.get_layer('conv2d_6')
    grads = K.gradients(test_img, last_conv_layer.output)[0]

    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    message = K.print_tensor(pooled_grads, message='pool_grad = ')
    iterate = K.function([model.input, K.learning_phase()],
                         [message, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([org_img.reshape(-1, 64, 64, 1), 0])
    for i in range(conv_layer_output_value.shape[2]):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    if normalise:
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)
    return heatmap

In [ ]:
def plot_heatmap(heatmap, loc_map):

    fig = plt.figure(figsize=(16, 8))

    grid = ImageGrid(fig, 111,  # as in plt.subplot(111)
                     nrows_ncols=(1, 2),
                     axes_pad=0.15,
                     share_all=True,
                     )

    # Add data to image grid
    im = grid[0].imshow(heatmap)
    im = grid[1].imshow(loc_map)
    plt.show()


## Calculate and plot heatmap

In [ ]:
num = -1

In [ ]:
heatmap = return_heatmap(model, test[num])

In [ ]:
plot_heatmap(heatmap, loc_map[num])